In [3]:
from IPython.display import clear_output

In [4]:

# Download the required libraries (needed when running outside colab where the environment doesn't come pre-loaded with libraries)

%pip install torch
%pip install torchvision
%pip install matplotlib

clear_output()

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from torchvision.datasets import MNIST
from torchvision.transforms.functional import to_tensor, to_pil_image, resize
from torchvision import transforms

from torch.utils.data import DataLoader
from torch.optim import Adam

import matplotlib.pyplot as plt

ImportError: DLL load failed while importing _C: The specified module could not be found.

# Sign Langauge Problem:

## Intution: The AI is so powrful and important because of its various applications on most if not all the different fields. As a result, we want to help in solving the issue of not understanding sign languages.

## - This notebook contains a detailed implementaiton of a NN uisng Pytorch.

### 1. Data set Details:

- The provided data set is ArASL (Arabic Alphabets Sign Language Dataset). It is as its name says, an arabic sighn langauge for Arabic alphabet. It has a total of 54049 images and their corrssponding labels.

### 2. Size of data:
- Length of train_dataset is 43239,
- Length of val_dataset is 10810

### 3. Labels representations:
- Each sample has a label, which can be one of the 32 classes.
It consist of 32 classes for the alphabet.
- The classes values are integers from 0 up to 31.


### 4. The mapping details:

- Each number represents a charcter. You can see the dictionary in the variable "mapping".

<br/>
-- You can see the mapping in the following:

0: 'seen', 1: 'zay', 2: 'aleff', 3: 'dal', 4: 'ta', 5: 'yaa', 6: 'fa', 7: 'ya', 8: 'khaa', 9: 'nun', 10: 'ha', 11: 'toot', 12: 'taa', 13: 'ra', 14: 'kaaf', 15: 'jeem', 16: 'laam', 17: 'la', 18: 'dhad', 19: 'dha', 20: 'waw', 21: 'meem', 22: 'al', 23: 'sheen', 24: 'haa', 25: 'thaa', 26: 'saad', 27: 'ghain', 28: 'ain', 29: 'thal', 30: 'gaaf', 31: 'bb'

### 5. Refrence:
Latif, G., Mohammad, N., Alghazo, J., AlKhalaf, R., & AlKhalaf, R. (2019). ARASL: Arabic Alphabets Sign Language Dataset. *Data in Brief*, 23, 103777. https://doi.org/10.1016/j.dib.2019.103777

## Downloading the dataset

### Run the following cells to download the MNIST dataset.

In [ ]:
!gdown https://data.mendeley.com/public-files/datasets/y7pckrw6z2/files/1efa0d6b-4d7f-4f58-9584-08f0488279ee/file_downloaded

In [ ]:
import os

def delete_folder(folder_path):
    try:
        # Get the list of files and subdirectories in the folder
        for item in os.listdir(folder_path):
            item_path = os.path.join(folder_path, item)

            # If it's a file, delete it
            if os.path.isfile(item_path):
                os.remove(item_path)
            # If it's a directory, recursively call delete_folder
            elif os.path.isdir(item_path):
                delete_folder(item_path)

        # Remove the empty folder
        os.rmdir(folder_path)
        print(f"Folder '{folder_path}' and its contents deleted successfully.")
    except FileNotFoundError:
        print(f"Folder '{folder_path}' not found.")
    except PermissionError:
        print(f"Permission error: Unable to delete folder '{folder_path}'.")
    except Exception as e:
        print(f"An error occurred: {e}")


delete_folder('ArASL_Database_54K_Final')

!unzip file_downloaded
clear_output()

## Just run these two cells. You are not supposed to explore them.

In [ ]:
import os

def rename_folders_and_create_mapping(folder_path):
    # Get the list of folders in the specified path
    folders = [folder for folder in os.listdir(folder_path) if os.path.isdir(os.path.join(folder_path, folder))]

    # Create a mapping from original folder names to numbers
    folder_mapping = {folder: i for i, folder in enumerate(folders)}

    # Rename the folders in-place and store the original names in the mapping
    for original_folder, number in folder_mapping.items():
        new_folder_name = str(number)
        new_folder_path = os.path.join(folder_path, new_folder_name)

        # Rename the folder
        os.rename(os.path.join(folder_path, original_folder), new_folder_path)

    return folder_mapping

folder_path = 'ArASL_Database_54K_Final'

# Create the folder mapping and rename folders
mapping = rename_folders_and_create_mapping(folder_path)

# Print the folder mapping
print("Folder Mapping:")
print(mapping)


In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import os

class CustomDataset(Dataset):
    def __init__(self, root_folder, transform=None, target_size=(28, 28)):
        self.root_folder = root_folder
        self.transform = transform
        self.target_size = target_size

        # Get the list of image files
        self.image_files = []
        self.image_labels = []

        for root, dirs, files in os.walk(root_folder):
            for file in files:
                if file.lower().endswith('.jpg'):
                    self.image_files.append(os.path.join(root, file))
                    self.image_labels.append(int(os.path.basename(root)))

    def __len__(self):
        return len(self.image_labels)

    def __getitem__(self, idx):
        img_path = self.image_files[idx]
        label = self.image_labels[idx]

        # Convert label to tensor
        # print("Label", label, type(label))
        label = torch.tensor(label)

        # Open the image
        with Image.open(img_path) as img:
            # Convert the image to grayscale
            img = img.convert('L')

            # Resize the image
            img = img.resize(self.target_size)

            # Apply additional transformations if specified
            if self.transform:
                img = self.transform(img)

            return (img, label)

# Define the root folder and output folder
root_folder_path = 'ArASL_Database_54K_Final'


# Define transformations (resize to 28x28 and convert to tensor)
data_transform = transforms.Compose([
    transforms.Resize((48, 48)),
    transforms.ToTensor(),
])

# Create an instance of the custom dataset
custom_dataset = CustomDataset(root_folder_path, transform=data_transform)

from torch.utils.data import random_split

# Define the size of the training and validation sets
total_size = len(custom_dataset)
train_size = int(0.8 * total_size)  # 80% for training
val_size = total_size - train_size  # 20% for validation

# Split the dataset
train_dataset, val_dataset = random_split(custom_dataset, [train_size, val_size])

mapping = {v:k for k,v in mapping.items()}

## Exploring the data

#### Here is the mapping for each class and its encoding. In addition to the train_dataset and val_dataset

In [ ]:
train_dataset # Contains the training ArASL_Database_54K_Final dataset (80%)

val_dataset  # Contains the validating ArASL_Database_54K_Final dataset (20%)


print("The mapping between the letters and the encoding: \n", mapping)

# Check the lengths of train_dataset and val_dataset.
print(len(train_dataset))
print(len(val_dataset))

In [ ]:
batch_size = 64 # To group each k samples together.

# DataLoaders simplify the job of grouping the samples into batches.
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)  # no need to shuffle validation data

## Let's visualize an image.

In [ ]:
random_img_idx = 30 # Write any random index (between 0 and 59999)

image = train_dataset[random_img_idx][0]  # 0 for image part in (image, label) tuple.
label = train_dataset[random_img_idx][1]  # 1 for label part

print("The image label:", label.item(), mapping[label.item()])

plt.imshow(image.reshape(image.shape[1], image.shape[1]), cmap='gray')
image.shape

### Create a loop to show 10 different images ranodmly

In [ ]:
import numpy as np
for _ in range(10):
    random_img_idx = np.random.randint(len(train_dataset))

    image = train_dataset[random_img_idx][0]
    label = train_dataset[random_img_idx][1]
    print("The image label:", label.item(), mapping[label.item()])

    plt.figure()
    plt.imshow(image.reshape(image.shape[1], image.shape[1]), cmap='gray')

### Create a NN uisng Pytorch. Then, train the model. Play with the layers to get the best model on the validation data, try to reach a 90% accuracy. At least, make 3 different layers. Also, try different activation functions.

In [ ]:
class NN(nn.Module):
    def __init__(self, ni, nh, no):
        super(NN, self).__init__()

        self.layer1 = nn.Linear(ni, nh)
        self.layer2 = nn.Linear(nh, nh//4)
        self.layer3 = nn.Linear(nh//4, no)

        self.hidden_activation = nn.ReLU()
        self.out_activation = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.hidden_activation(self.layer1(x))
        x = self.hidden_activation(self.layer2(x))
        x = self.out_activation(self.layer3(x))

        return x


In [ ]:
LENGTH = 48
epochs = 10
lr = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'

model = NN(LENGTH * LENGTH, 1024, 32)

optimizer = Adam(model.parameters())
loss_func = nn.CrossEntropyLoss()

model = model.to(device)
print(f'Device: {device}')

In [ ]:
train_losses = []
val_losses = []

for epoch_num in range(epochs):
    model.train()

    epoch_losses = 0
    val_epoch_losses = 0
    correct_pred = 0

    for batch_x, batch_y in train_loader:
        batch_x = batch_x.view(-1, LENGTH * LENGTH).to(device)
        batch_y = batch_y.to(device)

        batch_y_probs = model(batch_x)

        loss = loss_func(batch_y_probs, batch_y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        epoch_losses += loss.item() * len(batch_y)

    epoch_loss = epoch_losses/len(train_loader.dataset)
    train_losses.append(epoch_loss)

    with torch.no_grad():
        model.eval()

        for val_batch_x, val_batch_y in val_loader:
            val_batch_x = val_batch_x.view(-1, LENGTH*LENGTH).to(device)
            val_batch_y = val_batch_y.to(device)

            val_batch_y_probs = model(val_batch_x)

            loss = loss_func(val_batch_y_probs, val_batch_y)

            val_batch_y_preds = val_batch_y_probs.argmax(dim=1)

            val_epoch_losses += loss.item() * len(val_batch_y)
            correct_pred += (val_batch_y==val_batch_y_preds).sum().item()
        val_epoch_loss = val_epoch_losses/len(val_loader.dataset)
        val_losses.append(val_epoch_loss)
    print(f'Epoch: {epoch_num}, train_loss={epoch_loss}, val_loss={val_epoch_loss}. labelled {correct_pred}/{len(val_loader.dataset)} correctly ({correct_pred/len(val_loader.dataset)*100}% accuracy)')
print(f'Training complete on device {device}. Change device variable and run again to see the difference.')